In [6]:
!pip install matplotlib numpy ipywidgets
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatLogSlider, Checkbox, IntSlider
from scipy.fftpack import fftshift, ifftshift, fft2, ifft2
from PIL import Image
%matplotlib inline


In [7]:
def load_image(image_path):
    """
    Load an image, convert it to grayscale, and return it as a NumPy array.
    
    Args:
        image_path (str): Path to the image file.

    Returns:
        numpy.ndarray: 2D array representing the grayscale image.
    """
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    return np.array(img)

def image_to_kspace(image):
    """
    Compute the 2D Fourier Transform of the image and shift the zero frequency 
    component to the center of the spectrum (k-space).
    
    Args:
        image (numpy.ndarray): 2D grayscale image.
    
    Returns:
        numpy.ndarray: k-space representation of the image.
    """
    return fftshift(fft2(image))

def create_circular_mask(kspace, radius, invert=False):
    """
    Create a circular mask to apply to k-space based on a given radius.
    
    Args:
        kspace (numpy.ndarray): k-space (Fourier domain) of the image.
        radius (int): Radius of the mask.
        invert (bool): Whether to invert the mask (True retains values inside the circle, 
                       False retains values outside the circle).
    
    Returns:
        numpy.ndarray: Binary mask with the same shape as k-space.
    """
    rows, cols = kspace.shape
    center_row, center_col = rows // 2, cols // 2
    
    # Create a distance map from the center of the k-space
    Y, X = np.ogrid[:rows, :cols]
    dist_from_center = np.sqrt((X - center_col)**2 + (Y - center_row)**2)
    
    # Mask: True for pixels outside the radius, False for inside
    mask = dist_from_center > radius
    
    # Optionally invert the mask
    if invert:
        mask = ~mask
    
    return mask

def plot_image_kspace(image, kspace, radius=0, invert=False):
    """
    Plot the original image, k-space, and the reconstructed image after applying 
    a mask in k-space.
    
    Args:
        image (numpy.ndarray): Original grayscale image.
        kspace (numpy.ndarray): k-space representation of the image.
        radius (int): Radius of the circular mask applied to k-space.
        invert (bool): Whether to invert the mask.
    """
    # Apply circular mask to k-space
    mask = create_circular_mask(kspace, radius, invert)
    masked_kspace = kspace * mask
    
    # Reconstruct the image from masked k-space (Inverse FFT)
    reconstructed_image = np.abs(ifft2(ifftshift(masked_kspace)))
    
    # Plotting the original image, k-space, and reconstructed image
    fig, ax = plt.subplots(1, 3, figsize=(15, 30))  # 3 rows, 1 column

    # Display the original image
    ax[0].imshow(image, cmap='gray')
    ax[0].set_title("Original Image", fontsize=13)
    ax[0].axis('off')
    
    # Display masked k-space with log scaling
    kspace_display = np.log(1 + np.abs(masked_kspace))
    ax[1].imshow(kspace_display, cmap='gray')
    ax[1].set_title(f"K-space (Masking Radius: {radius})", fontsize=13)
    ax[1].axis('off')
    
    # Display the reconstructed image from the masked k-space
    ax[2].imshow(reconstructed_image, cmap='gray')
    ax[2].set_title("Reconstructed Image", fontsize=13)
    ax[2].axis('off')

    plt.show()

def interactive_plot(radius=0, invert=False):
    """
    Interactive function to update the plot based on user inputs for the mask 
    radius and whether the mask is inverted.
    
    Args:
        radius (int): Radius of the circular mask applied to k-space.
        invert (bool): Whether to invert the mask.
    """
    image = load_image(image_path)
    kspace = image_to_kspace(image)
    plot_image_kspace(image, kspace, radius, invert)

def visualize_kspace(image_path):
    """
    Visualize the k-space and image reconstruction using an interactive plot that 
    allows adjusting the mask radius and mask inversion.
    
    Args:
        image_path (str): Path to the image file to be loaded.
    """
    # Create interactive widgets for mask radius and inversion
    interact(interactive_plot,
             radius=IntSlider(min=0, max=800, step=1, value=0, description="Mask Radius"),
             invert=Checkbox(value=False, description="Invert Mask"))


In [8]:
image_path = r''

# Call the visualization function to see the output
visualize_kspace(image_path)

interactive(children=(IntSlider(value=0, description='Mask Radius', max=800), Checkbox(value=False, descriptio…